### Data Wrangling

This notebook is divided into four sections, one for each dataset. Following steps are performed on each section,

1. Load dataset
2. Data wrangling
3. Export dataset

In [1]:
# Import all required packages

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import missingno as ms


### Load dataset 1

Holiday performance: df_holidays


In [2]:
# load holiday performance xls file

xls = pd.ExcelFile(r'C:\Users\Adi\Desktop\Data_Science\Capstone\Capstone_2\Data\Holidays.xlsx') 

# print list of all sheets in the file
print(xls.sheet_names)

# read all sheets from file into dict
sheets = {}
for sheet in xls.sheet_names:
    sheets[sheet] = xls.parse(sheet)

# create separate dataframe for holiday dates from first sheet
holiday_list = sheets[xls.sheet_names[0]]
holiday_list.columns = [x.lower() for x in holiday_list.columns] # set column names to lower case

# add holiday name as new column in each sheet    
for sheet in xls.sheet_names[1:]: # leave the first sheet as it's a list of holiday dates
    sheets[sheet]['Holiday'] = sheet

# create df_holidays starting second sheet. first sheet is list of holiday dates
df_holidays = sheets[xls.sheet_names[1]]

# append remaining sheets in df_holidays
for sheet in xls.sheet_names[2:]:
    df_holidays = df_holidays.append(sheets[sheet])

['List', 'Presidents_day', 'Easter', 'Memorial_day', 'Independence_day', 'Labor_day', 'Thanksgiving', 'Winter_holiday']


In [3]:
# correct column names
df_holidays_column_names = ['year', 'total_flights', 'dep_ontime', 'dep_delayed', 'dep_cancel', 'arr_ontime', 'arr_delayed', 'arr_div', 'holiday']

# drop percentage of total columns. These can be calculated as column over total value
df_holidays.drop(columns=['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 13'], inplace=True)

# update columns names from list
df_holidays.columns = df_holidays_column_names


In [4]:
# merge df_holidays with holiday_list to get date and period
df_holidays = df_holidays.merge(holiday_list, on=['holiday', 'year'], how='left')

In [5]:
# final df_holidays
print(df_holidays.info())
df_holidays.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 69
Data columns (total 12 columns):
year               70 non-null int64
total_flights      70 non-null object
dep_ontime         70 non-null object
dep_delayed        70 non-null object
dep_cancel         70 non-null object
arr_ontime         70 non-null object
arr_delayed        70 non-null object
arr_div            70 non-null object
holiday            70 non-null object
date of holiday    70 non-null datetime64[ns]
start date         70 non-null datetime64[ns]
end date           70 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(8)
memory usage: 7.1+ KB
None


,year,total_flights,dep_ontime,dep_delayed,dep_cancel,arr_ontime,arr_delayed,arr_div,holiday,date of holiday,start date,end date
0,2009,105227,85420,18588,1219,82028,21799,181,Presidents_day,2009-02-16,2009-02-12,2009-02-17
1,2010,103994,73339,23249,7406,71299,24935,354,Presidents_day,2010-02-15,2010-02-11,2010-02-16
2,2011,98691,74056,22245,2390,72013,24020,268,Presidents_day,2011-02-21,2011-02-17,2011-02-22
3,2012,80518,70211,9964,343,69675,10390,110,Presidents_day,2012-02-20,2012-02-16,2012-02-21
4,2013,102244,86573,15023,648,85807,15676,113,Presidents_day,2013-02-18,2013-02-14,2013-02-19


In [6]:
# export dataset

df_holidays.to_csv('Dataset_export\holidays.csv', index=False)

### Load dataset 2

Delay causes: df_delay_cause


In [7]:
# Read dataset
df = pd.read_csv(r'C:\Users\Adi\Desktop\Data_Science\Capstone\Capstone_2\Data\Delay_cause.csv')

# drop empty column
df.drop(columns=['Unnamed: 21'], inplace=True)

# df_delay_cause with data from 2017 and 2018 only because analysis is for flights starting 2017
df_delay_cause = df[(df.year == 2017) | (df.year == 2018)]
df_delay_cause.reset_index(inplace=True) # reset index to start from 0

# update column names with correct spelling
df_delay_cause.columns = ['index', 'year', 'month', 'carrier', 'carrier_name', 'airport',
       'airport_name', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct',
       'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled',
       'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay']

In [8]:
# display df_delay_cause
print(df_delay_cause.info())
df_delay_cause.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24195 entries, 0 to 24194
Data columns (total 22 columns):
index                  24195 non-null int64
year                   24195 non-null int64
month                  24195 non-null int64
carrier                24195 non-null object
carrier_name           24195 non-null object
airport                24195 non-null object
airport_name           24195 non-null object
arr_flights            24177 non-null float64
arr_del15              24170 non-null float64
carrier_ct             24177 non-null float64
weather_ct             24177 non-null float64
nas_ct                 24177 non-null float64
security_ct            24177 non-null float64
late_aircraft_ct       24177 non-null float64
arr_cancelled          24177 non-null float64
arr_diverted           24177 non-null float64
arr_delay              24177 non-null float64
carrier_delay          24177 non-null float64
weather_delay          24177 non-null float64
nas_delay              2417

,index,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,220115,2017,1,AA,American Airlines Inc.,ABQ,"Albuquerque, NM: Albuquerque International Sun...",125.0,31.0,16.26,...,0.00,10.36,1.0,0.0,1378.0,758.0,21.0,95.0,0.0,504.0
1,220116,2017,1,AA,American Airlines Inc.,ALB,"Albany, NY: Albany International",88.0,10.0,4.52,...,0.00,4.41,1.0,0.0,402.0,171.0,0.0,17.0,0.0,214.0
2,220117,2017,1,AA,American Airlines Inc.,AMA,"Amarillo, TX: Rick Husband Amarillo International",24.0,6.0,1.65,...,0.00,2.79,1.0,0.0,419.0,59.0,64.0,166.0,0.0,130.0
3,220118,2017,1,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",1112.0,204.0,72.58,...,1.98,65.02,24.0,6.0,11968.0,4237.0,108.0,2693.0,159.0,4771.0
4,220119,2017,1,AA,American Airlines Inc.,AUS,"Austin, TX: Austin - Bergstrom International",729.0,126.0,47.75,...,0.99,38.06,2.0,0.0,6816.0,3096.0,103.0,1419.0,110.0,2088.0


In [9]:
# export dataset

df_delay_cause.to_csv('Dataset_export\delay_cause.csv', index=False)

### Load dataset 3

National performance: df_national_performance

In [10]:
# Read dataset from xlsx file
xls = pd.ExcelFile(r'C:\Users\Adi\Desktop\Data_Science\Capstone\Capstone_2\Data\Airline_performance_month_year.xlsx')
df = xls.parse('Sheet')

# drop empty columns
df_national_performance = df.iloc[:,:9]

In [11]:
print(df_national_performance.info())
df_national_performance.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 9 columns):
Rank                           283 non-null int64
Year                           283 non-null int64
Month                          283 non-null int64
Percent On-Time Arrivals       283 non-null float64
Percent Late Arrivals          283 non-null float64
Percent Cancelled              283 non-null float64
Percent Diverted               283 non-null float64
Percent On-Time Departures     283 non-null float64
Number of Scheduled Flights    283 non-null int64
dtypes: float64(5), int64(4)
memory usage: 20.0 KB
None


,Rank,Year,Month,Percent On-Time Arrivals,Percent Late Arrivals,Percent Cancelled,Percent Diverted,Percent On-Time Departures,Number of Scheduled Flights
0,1,2009,11,88.59,10.74,0.54,0.14,89.03,509540
1,2,2017,11,88.27,11.30,0.31,0.13,87.93,454162
2,3,2002,9,87.95,11.07,0.86,0.12,89.99,429996
3,4,2015,10,86.97,12.36,0.50,0.17,87.09,486165
4,5,2003,4,86.85,11.84,1.20,0.12,89.16,527303


In [13]:
# export dataset

df_national_performance.to_csv(r'Dataset_export\national_performance.csv', index=False)

### Load dataset 4

Flight performance: frames
(from Jan 2017 to Jul 2018)

In [14]:
# list of years and months to be used throughout the code
# data available from Jan 2017 to Jul 2018

years = ['2017', '2018']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [15]:
# initialize empty dictonary which will hold 19 dataframes, Jan 2017 to July 2018
frames = {}

# read all files using pandas
for year in years:
    if year == '2017':
        for month in months:
            path = 'data/' + year + ' ' + month + '.csv'
            df = pd.read_csv(path, low_memory=False)
            frames[year + month] = df
    else:
        for month in months[:7]: # dataset available only for first 7 months of 2018
            path = 'data/' + year + ' ' + month + '.csv'
            df = pd.read_csv(path, low_memory=False)
            frames[year + month] = df

In [16]:
# list of columns not required for this analysis

drop_columns = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DOT_ID_Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDelay', 'WheelsOff', 'WheelsOn', 'DepDelayMinutes', 'DepartureDelayGroups', 'ArrDelay', 'ArrDelayMinutes', 'ArrivalDelayGroups', 'CRSElapsedTime', 'ActualElapsedTime', 'Flights', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime', 'AirTime', 'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime', 'DivArrDelay', 'DivDistance', 'Div1Airport', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn', 'Div1TotalGTime', 'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum', 'Div2Airport', 'Div2AirportID', 'Div2AirportSeqID', 'Div2WheelsOn', 'Div2TotalGTime', 'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum', 'Div3Airport', 'Div3AirportID', 'Div3AirportSeqID', 'Div3WheelsOn', 'Div3TotalGTime', 'Div3LongestGTime', 'Div3WheelsOff', 'Div3TailNum', 'Div4Airport', 'Div4AirportID', 'Div4AirportSeqID', 'Div4WheelsOn', 'Div4TotalGTime', 'Div4LongestGTime', 'Div4WheelsOff', 'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum', 'Unnamed: 109']

# keep columns 
# 'FlightDate', 'DayOfWeek', 'Reporting_Airline', 'Tail_Number', 'Dest', 'Origin', 'CRSDepTime', 'DepTime'
# 'DepDel15', 'DepTimeBlk', 'TaxiOut', 'TaxiIn', 'CRSArrTime', 'ArrTime'
# 'ArrDel15', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'Distance'
# 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'

In [17]:
# fill nan function replaces missing values for high impact columns i.e. departure and arrival time
# if column value cannot be missing for this analysis it is categorized as high impact

def fill_nan_high(df, strategy='drop'):
    
    if strategy == 'drop':
        
        # using strategy drop nan rows
        # less than 3% of total data
        
        df.dropna(subset=['DepTime'], inplace=True)
        df.dropna(subset=['ArrTime'], inplace=True)
    
    else:
        
        # replace missing data
        # explore more data to find correct groupby condition
        
        df['DepTime'] = df.groupby(['DoW', 'Reporting_Airline', 'Origin', 'Dest', 'DepTimeBlk'])\
                        ['DepTime'].transform(lambda x: x.fillna(x.mean()))

        df['ArrTime'] = df.groupby(['DoW', 'Reporting_Airline', 'Origin', 'Dest', 'ArrTimeBlk'])\
                        ['ArrTime'].transform(lambda x: x.fillna(x.mean()))

In [18]:
# convert columns to int

def int_columns(df):
    
    # list of columns 
    
    int_columns = ['CRSDepTime', 'DepTime', 'DepDel15', 'TaxiOut', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDel15', 'Cancelled', 'Diverted', 'Distance', 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
    
    for column in int_columns:
        df.loc[df[column].notnull(), column] = df.loc[df[column].notnull(), column].astype(int)

In [19]:
def fill_nan(df):

    # medium impact columns - missing data can be imputed with mean/median value

    nan_columns_medium = ['TaxiOut', 'TaxiIn']

    df['TaxiOut'] = df.groupby(['DayOfWeek', 'Origin', 'DepTimeBlk'])['TaxiOut']\
                      .transform(lambda x: x.fillna(x.mean()))
        
    df['TaxiIn'] = df.groupby(['DayOfWeek', 'Dest', 'ArrTimeBlk'])['TaxiIn']\
                     .transform(lambda x: x.fillna(x.mean()))

    # low impact - missing data can be replaced with 0
    # value 1 only if flag is valid
    
    nan_columns_low = ['CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
    
    for column in nan_columns_low:
        df[column].fillna(0, inplace=True)

In [20]:
# set datettime columns to correct format

def datetime_columns(df):
    
    # check if flightdate should be converted to datetime
    #df['FlightDate']
    
    time_columns = ['CRSDepTime', 'DepTime', 'ArrTime', 'CRSArrTime']
    
    for column in time_columns:

        # fill with zeros to make 4 digit long       
        df.loc[df[column].notnull(), column] = df.loc[df[column].notnull(), column]\
                                                 .astype('str').str.pad(4,'left','0')

        # convert 2400 to 0000 i.e. midnight
        df.loc[df[column] == '2400', column] = '0000'

        # split into hh:mm
        df.loc[df[column].notnull(), column] = df.loc[df[column].notnull(), column].str[0:2] + ':' + \
                                               df.loc[df[column].notnull(), column].str[2:4]

In [21]:
# derived columns

def column_operations(df):
    
    # set departure and arrival delay flag 1 for threshold more than 15 minutes
    
    FMT = '%H:%M'
    DepDel15 = df['DepTime'].apply(lambda x: datetime.strptime(x, FMT)) - \
               df['CRSDepTime'].apply(lambda x: datetime.strptime(x, FMT))
        
    ArrDel15 = df['ArrTime'].apply(lambda x: datetime.strptime(x, FMT)) - \
               df['CRSArrTime'].apply(lambda x: datetime.strptime(x, FMT))
    
    # condition if time difference cant be negative i.e. clock has to cross midnight
    # if tdelta.days < 0:
    #     tdelta = timedelta(days=0, seconds=tdelta.seconds, microseconds=tdelta.microseconds)    
        
    df['DepDel15'] = np.where(DepDel15.astype('timedelta64[m]') > 15, 1, 0)
    df['ArrDel15'] = np.where(DepDel15.astype('timedelta64[m]') > 15, 1, 0)

In [22]:
# how to subplot missing value matrix for all frames?
# ms.matrix(df)

In [23]:
df_list = []
df_list_cancelled = []

for key, value in frames.items():
    
    # insert wrangle functions here
    
    df = frames[key].loc[frames[key].Cancelled == 0, : ]            # split frames for non cancelled flights
    df_cancelled = frames[key].loc[frames[key].Cancelled == 1, : ]  # split frames for cancelled flights
    
    df.drop(columns=drop_columns, inplace=True) # drop unwanted columns
    df_cancelled.drop(columns=drop_columns, inplace=True)
    
    fill_nan_high(df, strategy='drop')          # fill nan for high impact columns 
    int_columns(df)                             # convert numberic columns to int
    fill_nan(df)                                # fill nan for medium and low impact columns
    datetime_columns(df)                        # correct datetime format
    column_operations(df)                       # update delay flags
    
    # check for null values
    
    #null = df.isnull().any().any()                          
    #print('Missing values in ' + key + ': ' + str(null))
    
    # create list of all dataframes
    path = 'Dataset_export/' + key + '.csv'
    df.to_csv(path, index=False)
    
    #df_list.append(df)
    df_list_cancelled.append(df_cancelled)

c:\miniconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
c:\miniconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
c:\miniconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
c:\miniconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning:

In [24]:
# concat list of dataframes to df_flights

#df_flights = pd.DataFrame()
df_cancelled = pd.DataFrame()

#df_flights = pd.concat(df_list, ignore_index=True)
df_cancelled = pd.concat(df_list_cancelled, ignore_index=True)

In [25]:
# export dataset

#df_flights.to_csv('Dataset_export\dataset_4.csv', index=False)
df_cancelled.to_csv('Dataset_export\cancelled_flights.csv', index=False)